In [ ]:
# import numpy as np
# from astropy.coordinates import SkyCoord, EarthLocation, AltAz
# from astropy.time import Time
# import astropy.units as u
# import csv
# import json
# import os
# from concurrent.futures import ThreadPoolExecutor
# import multiprocessing

# # Create the folder for output files if it doesn't exist
# output_folder = 'NAROO_crossmatch_All'
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# def save_results_to_csv(results, output_file):
#     with open(output_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Label', 'GAIA ID', 'Old RA', 'Old DEC', 'New RA', 'New DEC', 'Observation Time', 'Plate RA', 'Plate DEC', 'Field Size', 'Angular Distance'])
#         for label, stars in results.items():
#             for star in stars:
#                 writer.writerow([label, star['gaia_id'], star['old_ra'], star['old_dec'], star['new_ra'], star['new_dec'], star['observation_time'], star['plate_ra'], star['plate_dec'], star['field_size'], star['angular_distance']])
#     print(f"Saved results to {output_file}")

# def save_results_to_json(results, output_file):
#     with open(output_file, mode='w') as file:
#         json.dump(results, file, indent=4)
#     print(f"Saved results to {output_file}")

# def read_gaia_data(file_path):
#     with open(file_path, 'r') as f:
#         lines = f.readlines()
    
#     gaia_data = []
#     for line in lines[1:]:  # Skip the header
#         parts = line.split()
#         if len(parts) == 7:
#             gaia_data.append({
#                 'gaia_id': int(parts[0]),
#                 'mg': float(parts[1]),
#                 'bp_rp': float(parts[2]),
#                 'ra': float(parts[3]),
#                 'dec': float(parts[4]),
#                 'pmra': float(parts[5]),
#                 'pmdec': float(parts[6])
#             })
#     return gaia_data

# def read_observation_data(file_path):
#     with open(file_path, 'r') as f:
#         lines = f.readlines()
    
#     photographic_plates = []
#     for line in lines:
#         if line.startswith('#') or line.strip() == '':
#             continue  # Skip headers and empty lines
#         parts = line.split()
#         if len(parts) >= 7:
#             try:
#                 plate_id = parts[0]
#                 date_str = parts[2]
#                 time_str = parts[3] if len(parts) > 4 else None
#                 dec_sign = parts[4][0]  # First character of declination part
#                 dec_rest = parts[4][1:]
#                 ra_parts = parts[3].split(':')
#                 dec_parts = dec_rest.split(':')
                
#                 ra_deg = float(ra_parts[0]) + float(ra_parts[1])/60 + float(ra_parts[2])/3600
#                 dec_deg = float(dec_parts[0]) + float(dec_parts[1])/60 + float(dec_parts[2])/3600
#                 if dec_sign == '-':
#                     dec_deg = -dec_deg
                
#                 photographic_plates.append({
#                     'plate_labels': plate_id,
#                     'time': date_str,
#                     'ra': ra_deg,
#                     'dec': dec_deg,
#                     'field_size': float(parts[6])
#                 })
#             except ValueError:
#                 continue
#     return photographic_plates

# def calculate_historical_position(ra, dec, pmra, pmdec, observation_time):
#     palomar_observatory = EarthLocation(lat=33.3564*u.deg, lon=-116.865*u.deg, height=1706*u.m)
#     try:
#         time_obs = Time(observation_time)
#     except ValueError as e:
#         print(f"Error parsing time '{observation_time}': {e}")
#         return None, None, None, None

#     current_time = Time.now()
#     star = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs', obstime=current_time)
#     new_ra = star.ra.deg
#     new_dec = star.dec.deg
    
#     delta_years = (time_obs - current_time).to(u.year).value
#     ra_displacement = pmra * delta_years * u.mas.to(u.deg)
#     dec_displacement = pmdec * delta_years * u.mas.to(u.deg)
    
#     old_ra = star.ra + ra_displacement * u.deg
#     old_dec = star.dec + dec_displacement * u.deg
    
#     return old_ra.deg, old_dec.deg, new_ra, new_dec

# def is_within_field(star_ra, star_dec, plate_ra, plate_dec, field_size):
#     star = SkyCoord(ra=star_ra*u.deg, dec=star_dec*u.deg, frame='icrs')
#     plate_center = SkyCoord(ra=plate_ra*u.deg, dec=plate_dec*u.deg, frame='icrs')
#     separation = star.separation(plate_center).deg
#     return separation <= (field_size / 2), separation

# def process_gaia_data_chunk(gaia_data, photographic_plate):
#     results = {}
#     for star in gaia_data:
#         old_ra, old_dec, new_ra, new_dec = calculate_historical_position(star['ra'], star['dec'], star['pmra'], star['pmdec'], photographic_plate['time'])
#         if old_ra is None or old_dec is None:
#             continue
#         within_field, angular_distance = is_within_field(old_ra, old_dec, photographic_plate['ra'], photographic_plate['dec'], photographic_plate['field_size'])
#         if within_field:
#             label = photographic_plate['plate_labels']
#             if label not in results:
#                 results[label] = []
#             results[label].append({
#                 'gaia_id': star['gaia_id'],
#                 'old_ra': old_ra,
#                 'old_dec': old_dec,
#                 'new_ra': new_ra,
#                 'new_dec': new_dec,
#                 'observation_time': photographic_plate['time'],
#                 'plate_ra': photographic_plate['ra'],
#                 'plate_dec': photographic_plate['dec'],
#                 'field_size': photographic_plate['field_size'],
#                 'angular_distance': angular_distance
#             })
#     return results

# def chunked_process_and_save(gaia_data, photographic_plates, output_folder):
#     chunk_files_csv = []
#     chunk_files_json = []

#     # Determine the number of threads to use
#     total_cores = multiprocessing.cpu_count()
#     max_workers = max(1, total_cores - 2)  # Ensure at least 1 worker

#     with ThreadPoolExecutor(max_workers=max_workers) as executor:
#         futures = {executor.submit(process_gaia_data_chunk, gaia_data, plate): plate['plate_labels'] for plate in photographic_plates}

#         for future in futures:
#             results = future.result()
#             label = futures[future]
#             chunk_file_csv = os.path.join(output_folder, f'{label}.csv')
#             chunk_file_json = os.path.join(output_folder, f'{label}.json')
#             save_results_to_csv(results, chunk_file_csv)
#             save_results_to_json(results, chunk_file_json)
#             chunk_files_csv.append(chunk_file_csv)
#             chunk_files_json.append(chunk_file_json)

#     return chunk_files_csv, chunk_files_json

# def combine_csv_files(input_files, output_file):
#     header_saved = False
#     with open(output_file, 'w', newline='') as fout:
#         for filename in input_files:
#             with open(filename, 'r') as fin:
#                 header = next(fin)
#                 if not header_saved:
#                     fout.write(header)
#                     header_saved = True
#                 for line in fin:
#                     fout.write(line)

# # Example usage
# gaia_file_path = 'stars.txt'
# observation_file_path = 'possi-db-naroo.txt'

# gaia_data = read_gaia_data(gaia_file_path)
# photographic_plates = read_observation_data(observation_file_path)

# chunk_files_csv, chunk_files_json = chunked_process_and_save(gaia_data, photographic_plates, output_folder)

# final_output_file_csv = 'NAROO_obs.csv'
# combine_csv_files(chunk_files_csv, final_output_file_csv)

# print(f"All chunks saved. Results are combined and saved to {final_output_file_csv}")
# print(f"JSON chunks are saved in {output_folder}")

Saved results to NAROO_crossmatch_All/E0570.csv
Saved results to NAROO_crossmatch_All/E0570.json
Saved results to NAROO_crossmatch_All/O0570.csv
Saved results to NAROO_crossmatch_All/O0570.json
Saved results to NAROO_crossmatch_All/E0567.csv
Saved results to NAROO_crossmatch_All/E0567.json
Saved results to NAROO_crossmatch_All/O0567.csv
Saved results to NAROO_crossmatch_All/O0567.json
Saved results to NAROO_crossmatch_All/E0568.csv
Saved results to NAROO_crossmatch_All/E0568.json
Saved results to NAROO_crossmatch_All/O0568.csv
Saved results to NAROO_crossmatch_All/O0568.json
Saved results to NAROO_crossmatch_All/E1277.csv
Saved results to NAROO_crossmatch_All/E1277.json
Saved results to NAROO_crossmatch_All/O1277.csv
Saved results to NAROO_crossmatch_All/O1277.json
Saved results to NAROO_crossmatch_All/E1328.csv
Saved results to NAROO_crossmatch_All/E1328.json
Saved results to NAROO_crossmatch_All/O1328.csv
Saved results to NAROO_crossmatch_All/O1328.json
Saved results to NAROO_crossma

In [4]:
import numpy as np
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
import csv
import json
import os
from concurrent.futures import ThreadPoolExecutor
import multiprocessing

# Create the folder for output files if it doesn't exist
output_folder = 'NAROO_crossmatch_All'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def save_results_to_csv(results, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Label', 'GAIA ID', 'Old RA', 'Old DEC', 'New RA', 'New DEC', 'Observation Time', 'Plate RA', 'Plate DEC', 'Field Size', 'Angular Distance'])
        for label, stars in results.items():
            for star in stars:
                writer.writerow([label, star['gaia_id'], star['old_ra'], star['old_dec'], star['new_ra'], star['new_dec'], star['observation_time'], star['plate_ra'], star['plate_dec'], star['field_size'], star['angular_distance']])
    print(f"Saved results to {output_file}")

def save_results_to_json(results, output_file):
    with open(output_file, mode='w') as file:
        json.dump(results, file, indent=4)
    print(f"Saved results to {output_file}")

def read_gaia_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    gaia_data = []
    for line in lines[1:]:  # Skip the header
        parts = line.split()
        if len(parts) == 7:
            gaia_data.append({
                'gaia_id': int(parts[0]),
                'mg': float(parts[1]),
                'bp_rp': float(parts[2]),
                'ra': float(parts[3]),
                'dec': float(parts[4]),
                'pmra': float(parts[5]),
                'pmdec': float(parts[6])
            })
    return gaia_data

def read_observation_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    photographic_plates = []
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue  # Skip headers and empty lines
        parts = line.split()
        if len(parts) >= 7:
            try:
                plate_id = parts[0]
                date_str = parts[2]
                time_str = parts[3] if len(parts) > 4 else None
                dec_sign = parts[4][0]  # First character of declination part
                dec_rest = parts[4][1:]
                ra_parts = parts[3].split(':')
                dec_parts = dec_rest.split(':')
                
                ra_deg = float(ra_parts[0]) + float(ra_parts[1])/60 + float(ra_parts[2])/3600
                dec_deg = float(dec_parts[0]) + float(dec_parts[1])/60 + float(dec_parts[2])/3600
                if dec_sign == '-':
                    dec_deg = -dec_deg
                
                photographic_plates.append({
                    'plate_labels': plate_id,
                    'time': date_str,
                    'ra': ra_deg,
                    'dec': dec_deg,
                    'field_size': float(parts[6])
                })
            except ValueError:
                continue
    return photographic_plates

def calculate_historical_position(ra, dec, pmra, pmdec, observation_time):
    palomar_observatory = EarthLocation(lat=33.3564*u.deg, lon=-116.865*u.deg, height=1706*u.m)
    try:
        time_obs = Time(observation_time)
    except ValueError as e:
        print(f"Error parsing time '{observation_time}': {e}")
        return None, None, None, None

    current_time = Time.now()
    star = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs', obstime=current_time)
    new_ra = star.ra.deg
    new_dec = star.dec.deg
    
    delta_years = (time_obs - current_time).to(u.year).value
    ra_displacement = pmra * delta_years * u.mas.to(u.deg)
    dec_displacement = pmdec * delta_years * u.mas.to(u.deg)
    
    old_ra = star.ra + ra_displacement * u.deg
    old_dec = star.dec + dec_displacement * u.deg
    
    return old_ra.deg, old_dec.deg, new_ra, new_dec

def is_within_field(star_ra, star_dec, plate_ra, plate_dec, field_size):
    star = SkyCoord(ra=star_ra*u.deg, dec=star_dec*u.deg, frame='icrs')
    plate_center = SkyCoord(ra=plate_ra*u.deg, dec=plate_dec*u.deg, frame='icrs')
    separation = star.separation(plate_center).deg
    return separation <= (field_size / 2), separation

def process_gaia_data_chunk(gaia_data, photographic_plate):
    results = {}
    for star in gaia_data:
        old_ra, old_dec, new_ra, new_dec = calculate_historical_position(star['ra'], star['dec'], star['pmra'], star['pmdec'], photographic_plate['time'])
        if old_ra is None or old_dec is None:
            continue
        within_field, angular_distance = is_within_field(old_ra, old_dec, photographic_plate['ra'], photographic_plate['dec'], photographic_plate['field_size'])
        if within_field:
            label = photographic_plate['plate_labels']
            if label not in results:
                results[label] = []
            results[label].append({
                'gaia_id': star['gaia_id'],
                'old_ra': old_ra,
                'old_dec': old_dec,
                'new_ra': new_ra,
                'new_dec': new_dec,
                'observation_time': photographic_plate['time'],
                'plate_ra': photographic_plate['ra'],
                'plate_dec': photographic_plate['dec'],
                'field_size': photographic_plate['field_size'],
                'angular_distance': angular_distance
            })
    return results

def chunked_process_and_save(gaia_data, photographic_plates, output_folder):
    chunk_files_csv = []
    chunk_files_json = []

    # Determine the number of threads to use
    total_cores = multiprocessing.cpu_count()
    max_workers = max(1, total_cores - 2)  # Ensure at least 1 worker

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        for plate in photographic_plates:
            label = plate['plate_labels']
            csv_file = os.path.join(output_folder, f'{label}.csv')
            json_file = os.path.join(output_folder, f'{label}.json')
            if os.path.exists(csv_file) and os.path.exists(json_file):
                print(f"Skipping {label} as both files already exist.")
                continue
            futures[executor.submit(process_gaia_data_chunk, gaia_data, plate)] = plate['plate_labels']

        for future in futures:
            results = future.result()
            label = futures[future]
            chunk_file_csv = os.path.join(output_folder, f'{label}.csv')
            chunk_file_json = os.path.join(output_folder, f'{label}.json')
            save_results_to_csv(results, chunk_file_csv)
            save_results_to_json(results, chunk_file_json)
            chunk_files_csv.append(chunk_file_csv)
            chunk_files_json.append(chunk_file_json)

    return chunk_files_csv, chunk_files_json

def combine_csv_files(input_files, output_file):
    header_saved = False
    with open(output_file, 'w', newline='') as fout:
        for filename in input_files:
            with open(filename, 'r') as fin:
                header = next(fin)
                if not header_saved:
                    fout.write(header)
                    header_saved = True
                for line in fin:
                    fout.write(line)

# Example usage
gaia_file_path = 'stars.txt'
observation_file_path = 'possi-db-naroo.txt'

gaia_data = read_gaia_data(gaia_file_path)
photographic_plates = read_observation_data(observation_file_path)

chunk_files_csv, chunk_files_json = chunked_process_and_save(gaia_data, photographic_plates, output_folder)

final_output_file_csv = 'NAROO_obs.csv'
combine_csv_files(chunk_files_csv, final_output_file_csv)

print(f"All chunks saved. Results are combined and saved to {final_output_file_csv}")
print(f"JSON chunks are saved in {output_folder}")

Skipping E0570 as both files already exist.
Skipping O0570 as both files already exist.
Skipping E0567 as both files already exist.
Skipping O0567 as both files already exist.
Skipping E0568 as both files already exist.
Skipping O0568 as both files already exist.
Skipping E1277 as both files already exist.
Skipping O1277 as both files already exist.
Skipping E1328 as both files already exist.
Skipping O1328 as both files already exist.
Skipping E0692 as both files already exist.
Skipping O0692 as both files already exist.
Skipping E0727 as both files already exist.
Skipping O0727 as both files already exist.
Skipping E1340 as both files already exist.
Skipping O1340 as both files already exist.
Skipping E1363 as both files already exist.
Skipping O1363 as both files already exist.
Skipping E0775 as both files already exist.
Skipping O0775 as both files already exist.
Skipping E0802 as both files already exist.
Skipping O0802 as both files already exist.
Skipping E1216 as both files alr

In [7]:
%pwd
%cd "/home/dutta/Downloads/Summer'24/NAROO_crossmatch_All"

/home/dutta/Downloads/Summer'24/NAROO_crossmatch_All


In [12]:
import json
import os

def combine_json_files(input_folder, output_file):
    combined_results = {}

    # List all JSON files in the input folder
    input_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]

    for filename in input_files:
        file_path = os.path.join(input_folder, filename)
        with open(file_path, 'r') as fin:
            data = json.load(fin)
            combined_results.update(data)

    with open(output_file, 'w') as fout:
        json.dump(combined_results, fout, indent=4)
    print(f"Combined JSON saved to {output_file}")

# Specify the input folder and the final combined JSON output file
input_folder = '.'
final_output_file_json = 'NAROO_obs.json'

# Combine all JSON files in the input folder into one
combine_json_files(input_folder, final_output_file_json)

Combined JSON saved to NAROO_obs.json
